In [1]:
import pandas as pd
import numpy as np
import sklearn
import pyreadr

#statistics
from scipy.stats import chi2_contingency, ttest_ind

# import cudf #gpu-powered DataFrame (Pandas alternative)

#imbalance handling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, RepeatedEditedNearestNeighbours
from imblearn.pipeline import Pipeline

#preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler


#internal validation
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold, cross_val_score, GridSearchCV, PredefinedSplit, train_test_split

#performance metrices
from sklearn.metrics import make_scorer, confusion_matrix, classification_report, f1_score, balanced_accuracy_score, matthews_corrcoef, auc, average_precision_score, roc_auc_score, balanced_accuracy_score, roc_curve, accuracy_score

#Models selection
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
# from cuml.svm import SVC #gpu-powered SVM

#Tree pruning
from sklearn.tree._tree import TREE_LEAF


#save and load trained model
import pickle

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree

from collections import Counter
import os
# import torch

random_state = 42

In [2]:
gridSearch_seq, crossVal_seq, internalEvaluation_seq, externalEvaluation_seq = pickle.load(open('../Clean_data/sequence_dataset_DF_27102024.sav', 'rb'))

#fix BTS seqs
bts_vars = gridSearch_seq.columns[gridSearch_seq.columns.str.contains('BTS')].values
for var in bts_vars:
    gridSearch_seq[var] = gridSearch_seq[var].apply(lambda x: int(x))
    externalEvaluation_seq[var] = externalEvaluation_seq[var].apply(lambda x: int(x))
    crossVal_seq[var] = crossVal_seq[var].apply(lambda x: int(x))
    internalEvaluation_seq[var] = internalEvaluation_seq[var].apply(lambda x: int(x))

In [3]:
gridSearchData, crossValData, internalEvaluationData, externalEvaluationData = pickle.load(open('../Clean_data/dataset_2vs1_25102024.sav', 'rb'))

In [4]:
crossVal_seq.drop(columns='outcome_12months', inplace=True)
crossValData = crossValData.merge(crossVal_seq, on='patid', how='inner')

gridSearch_seq.drop(columns='outcome_12months', inplace=True)
gridSearchData = gridSearchData.merge(gridSearch_seq, on='patid', how='inner')

internalEvaluation_seq.drop(columns='outcome_12months', inplace=True)
internalEvaluationData = internalEvaluationData.merge(internalEvaluation_seq, on='patid', how='inner')

externalEvaluation_seq.drop(columns='outcome_12months', inplace=True)
externalEvaluationData = externalEvaluationData.merge(externalEvaluation_seq, on='patid', how='inner')

In [5]:
print(gridSearchData.shape)
print(externalEvaluationData.shape)

(29432, 1447)
(19678, 1447)


In [6]:
#features
tabular_vars = ['sex', 'rhinitis', 'cardiovascular', 'heartfailure', 'psoriasis', 'anaphylaxis', 'diabetes', 'ihd', 'anxiety', 'eczema', 
                'nasalpolyps', 'BMI_cat_normal', 'BMI_cat_not recorded', 'BMI_cat_obese', 'BMI_cat_overweight', 'BMI_cat_underweight', 
                'ethnic_group_Asian', 'ethnic_group_Black', 'ethnic_group_Mixed', 'ethnic_group_Other', 'ethnic_group_White', 
                'ethnic_group_not recorded', 'smokingStatus_current', 'smokingStatus_former', 'smokingStatus_never', 'imd_decile_0', 
                'imd_decile_1', 'imd_decile_2', 'imd_decile_3', 'imd_decile_4', 'imd_decile_5', 'imd_decile_6', 'imd_decile_7', 'imd_decile_8', 
                'imd_decile_9', 'imd_decile_10', 'CharlsonScore_0.0', 'CharlsonScore_1.0', 'CharlsonScore_2.0', 'CharlsonScore_3.0', 'CharlsonScore_4.0', 
                'CharlsonScore_5.0', 'CharlsonScore_6.0', 'CharlsonScore_7.0', 'CharlsonScore_8.0', 'CharlsonScore_9.0', 'CharlsonScore_10.0', 
                'CharlsonScore_11.0', 'CharlsonScore_12.0', 'PEFStatus_60-80', 'PEFStatus_less than 60', 'PEFStatus_more than 80', 'PEFStatus_not recorded', 
                'EosinophilLevel_high', 'EosinophilLevel_normal', 'EosinophilLevel_not recorded', 'BTS_step_0', 'BTS_step_1', 'BTS_step_2', 'BTS_step_3', 
                'BTS_step_4', 'BTS_step_5', 'DeviceType_BAI', 'DeviceType_DPI', 'DeviceType_NEB', 'DeviceType_not recorded', 'DeviceType_pMDI', 
                'PriorEducation_No', 'PriorEducation_Yes', 'age', 
                ]
seq_vars = gridSearch_seq.columns.values.tolist()
feature_columns = tabular_vars+seq_vars[1:]

In [10]:
X_gridSearch = gridSearchData[feature_columns]
X_crossVal = crossValData[feature_columns]
X = pd.concat([X_gridSearch, X_crossVal])
X.reset_index(drop=True, inplace=True)
X_internalVal = internalEvaluationData[feature_columns]
X_externalVal = externalEvaluationData[feature_columns]


In [11]:
def build_models (X_train, y_train, target_outcome, params_dict, model_folder, fold):
    models = [] #list to store all the models
    print("Building models . . . .")

    # #LR
    # model = 'LR'
    # params = params_dict[(params_dict['outcome']==target_outcome)&(params_dict['model']==model)]['params'].tolist()[0]
    # # params = eval(params)
    # print(params)
    # lr_model = LogisticRegression(class_weight='balanced', C = params['C'], max_iter=params['max_iter'], solver=params['solver'], random_state=random_state)
    # lr_model.fit(X_train,y_train)
    # pickle.dump(lr_model, open(model_folder+ target_outcome + '_'+ model + str(fold) + '.sav', 'wb'))
    # models.append([model + str(fold), target_outcome, y_train.value_counts()[1]/y_train.value_counts()[0]]) 
    # print("LR done")

    # #Lasso
    # model = 'Lasso'
    # params = params_dict[(params_dict['outcome']==target_outcome)&(params_dict['model']==model)]['params'].tolist()[0]
    # # params = eval(params)
    # print(params)
    # lasso_model = LogisticRegression(class_weight='balanced',  C = params['C'], max_iter=params['max_iter'], penalty='l1', solver=params['solver'], random_state=random_state) #only the LIBLINEAR and SAGA (added in v0.19) solvers handle the L1 penalty
    # lasso_model.fit(X_train, y_train)
    # pickle.dump(lasso_model, open(model_folder+ target_outcome + '_'+ model + str(fold) + '.sav', 'wb'))
    # models.append([model + str(fold), target_outcome, y_train.value_counts()[1]/y_train.value_counts()[0]])
    # print("Lasso done")
    
    # #Elastics
    # model = 'ElasticNet'
    # params = params_dict[(params_dict['outcome']==target_outcome)&(params_dict['model']==model)]['params'].tolist()[0]
    # # params = eval(params)
    # print(params)
    # elastics_model = LogisticRegression(class_weight='balanced', solver='saga', l1_ratio=params['l1_ratio'], max_iter=params['max_iter'],  penalty = 'elasticnet', random_state=random_state)
    # elastics_model.fit(X_train, y_train)
    # pickle.dump(elastics_model, open(model_folder+ target_outcome + '_'+ model + str(fold) + '.sav', 'wb'))
    # models.append([model + str(fold), target_outcome, y_train.value_counts()[1]/y_train.value_counts()[0]])
    # print("Elastics done")


    # #DT
    # model = 'DT'
    # params = params_dict[(params_dict['outcome']==target_outcome)&(params_dict['model']==model)]['params'].tolist()[0]
    # # params = eval(params)
    # print(params)
    # dt_model = DecisionTreeClassifier(class_weight='balanced', max_depth=params['max_depth'], criterion=params['criterion'], splitter=params['splitter'], random_state=random_state)
    # dt_model.fit(X_train, y_train)
    # pickle.dump(dt_model, open(model_folder+ target_outcome + '_'+ model + str(fold) + '.sav', 'wb'))    
    # models.append([model + str(fold), target_outcome, y_train.value_counts()[1]/y_train.value_counts()[0]])
    # print("DT done")

    # #RF
    # model = 'RF'
    # params = params_dict[(params_dict['outcome']==target_outcome)&(params_dict['model']==model)]['params'].tolist()[0]
    # # params = eval(params)
    # print(params)
    # rf_model = RandomForestClassifier(class_weight='balanced', max_depth=params['max_depth'], 
    #                                   criterion=params['criterion'], n_estimators=params['n_estimators'], 
    #                                   min_samples_split=params['min_samples_split'],
    #                                   min_samples_leaf=params['min_samples_leaf'],
    #                                   max_features=params['max_features'],
    #                                   bootstrap=params['bootstrap'], 
    #                                   random_state=random_state)
    # rf_model.fit(X_train, y_train)
    # pickle.dump(rf_model, open(model_folder+ target_outcome + '_'+ model + str(fold) + '.sav', 'wb'))     
    # models.append([model + str(fold), target_outcome, y_train.value_counts()[1]/y_train.value_counts()[0]])
    # print("RF done")

    #XGB
    model = 'XGB'
    params = params_dict[(params_dict['outcome']==target_outcome)&(params_dict['model']==model)]['params'].tolist()[0]
    # params = eval(params)
    print(params)
    scale_pos_ratio = y_train.value_counts()[0]/y_train.value_counts()[1]
    xgb_model = xgb.XGBClassifier(objective ='binary:logistic', tree_method = "hist", 
                                  n_estimators=params['n_estimators'],
                                  max_depth=params['max_depth'],
                                  learning_rate=params['learning_rate'],
                                  reg_alpha=params['reg_alpha'],
                                  reg_lambda=params['reg_lambda'],
                                  subsample=params['subsample'],
                                  colsample_bytree=params['colsample_bytree'],
                                  scale_pos_weight=params['scale_pos_weight'],
                                  device = "cuda", 
                                  verbosity = 3,
                                  importance_type = 'gain', random_state=random_state)
    # xgb_model = xgb.XGBClassifier(objective ='binary:logistic', learning_rate = 0.001, tree_method='gpu_hist', gpu_id=0,  verbosity = 0, random_state = 1234)
    xgb_model.fit(X_train,y_train)
    pickle.dump(xgb_model, open(model_folder+ target_outcome + '_'+ model + str(fold) + '.sav', 'wb')) 
    models.append([model + str(fold),  target_outcome, y_train.value_counts()[1]/y_train.value_counts()[0]])
    print("XGB done")
    
    return models
    # return [xgb_model]

In [12]:
def summariseResult (testX, testY, model):
    preds = model.predict_proba(testX)
    preds = [x[1] for x in preds]
    # tn, fp, fn, tp = confusion_matrix(testY, preds).ravel()
    # specificity = tn / (tn+fp)
    # sensitivity = tp / (tp+fn)
    # ppv = 100*tp/(tp+fp)
    # npv = 100*tn/(fn+tn)
    # acc = accuracy_score(testY, preds)
    # f1score = f1_score(testY, preds, average = 'binary')
    # balanceacc = balanced_accuracy_score(testY, preds)
    fpr, tpr, thresholds = roc_curve(testY, preds, pos_label=1)
    # aucscore = auc(fpr, tpr)
    aucscore = roc_auc_score(testY, preds)
    auprc = average_precision_score(testY, preds)
    # plot_confusion_matrix(model, testX, testY, cmap='viridis')  
    return np.round(aucscore,4), np.round(auprc,4)

In [13]:
params_dict = pd.read_csv('../MODELS/BS_result_new.csv')
def process_params(param_items, best_param):
    a = eval(param_items)
    b = eval(best_param)
    c = {}
    for key, value in zip(a,b):
        c[key] = value
    return c

params_dict['params'] = params_dict.apply(lambda x: dict(eval(x.best_param[11:])), axis=1)


In [16]:

#EXECUTE model training

summary_result_internalVal = []
summary_result_externalVal = []

cols = ['model_name', 'fold', 'outcome', 'class_ratio', 'auc', 'auprc']
model_folder = '../MODELS/TestResultCat/'
target_outcomes = ['outcome_12months']
# start_time = time.time()
for target_outcome in target_outcomes:
    models = pd.DataFrame(columns=['modelname', 'target_outcome', 'class_ratio'])
    print(target_outcome)
    y = pd.concat([gridSearchData[target_outcome], crossValData[target_outcome]])
    y.reset_index(drop=True, inplace=True)
    y_internalVal = internalEvaluationData[target_outcome]
    y_externalVal = externalEvaluationData[target_outcome]
    fold = 'LONG' #no fold for the FINAL MODEL - trained on crossval and gridsearch sets
    
    
    #Build models -> it can be commented if the models have been trained
    models_temp = pd.DataFrame(build_models(X, y, target_outcome, params_dict, model_folder, fold), columns=['modelname', 'target_outcome', 'class_ratio'])
    models = pd.concat([models,models_temp]).reset_index(drop=True)


    #evaluate model
    for modelname, target_outcome, classratio in models.values:
        # print('======================================================================')
        print(modelname)
        model = pickle.load(open(model_folder + target_outcome + '_'+ modelname + '.sav', 'rb'))       
        summary_result_internalVal.append((modelname, fold, target_outcome, classratio, ) + summariseResult (X_internalVal, y_internalVal, model) )       
        summary_result_externalVal.append((modelname, fold, target_outcome, classratio, ) + summariseResult (X_externalVal, y_externalVal, model) )       
        # torch.cuda.empty_cache()


summary_result_internalVal = pd.DataFrame(summary_result_internalVal, columns=cols)
summary_result_internalVal['model_num'] = summary_result_internalVal.index
# summary_result_internalVal.to_csv('../MODELS/internalValResultCat.csv', index_label=False, index=False)

summary_result_externalVal = pd.DataFrame(summary_result_externalVal, columns=cols)
summary_result_externalVal['model_num'] = summary_result_externalVal.index
# summary_result_externalVal.to_csv('../MODELS/externalValResultCat.csv', index_label=False, index=False)

outcome_12months
Building models . . . .
{'colsample_bytree': 0.5, 'learning_rate': 0.021211820714260574, 'max_depth': 2, 'n_estimators': 1000, 'reg_alpha': 10.0, 'reg_lambda': 0.1, 'scale_pos_weight': 9.691656942823805, 'subsample': 0.5}
[04:29:23] ======== Monitor (0): HostSketchContainer ========
[04:29:23] AllReduce: 0.015439s, 1 calls @ 15439us

[04:29:23] MakeCuts: 0.026672s, 1 calls @ 26672us

[04:29:23] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3
[04:29:23] DEBUG: /workspace/src/tree/updater_gpu_hist.cu:822: [GPU Hist]: Configure
[04:29:23] ======== Monitor (0):  ========
[04:29:23] InitCompressedData: 0.109395s, 1 calls @ 109395us

[04:29:37] ======== Monitor (0): Learner ========
[04:29:37] Configure: 0.00181s, 1 calls @ 1810us

[04:29:37] EvalOneIter: 0.00413s, 1000 calls @ 4130us

[04:29:37] GetGradient: 0.093982s, 1000 calls @ 93982us

[04:29:37] PredictRaw: 0.000993s, 1000 calls @ 993us

[04:29:37] UpdateOneIter: 12.3037s, 1000 calls @ 12303669us

[04:29

/tmp/ipykernel_19616/1827634542.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  models = pd.concat([models,models_temp]).reset_index(drop=True)


[04:29:37] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3
[04:29:37] DEBUG: /workspace/src/tree/updater_gpu_hist.cu:822: [GPU Hist]: Configure


In [17]:
summary_result_externalVal

,model_name,fold,outcome,class_ratio,auc,auprc,model_num
0,XGBLONG,LONG,outcome_12months,0.131724,0.7834,0.4633,0


In [18]:
summary_result_internalVal

,model_name,fold,outcome,class_ratio,auc,auprc,model_num
0,XGBLONG,LONG,outcome_12months,0.131724,0.8062,0.4474,0
